In [1]:
import h5py
import cantera as ct
import numpy as np

from pyrometheus.codegen.python import PythonCodeGenerator as pyro
from matplotlib import pyplot as plt

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

In [2]:
fuel = 'H2'
mech = {'H2': 'sandiego', 'C2H4': 'bfer', 'CH4': 'gri30'}
fuel_name = {'H2': 'hydrogen', 'C2H4': 'ethylene'}

In [3]:
equiv_ratio = 1

sol = ct.Solution(f'../mech/{mech[fuel]}.yaml')
pyro_gas = pyro.get_thermochem_class(sol)(np)
temp_c = 300
sol.TP = temp_c, ct.one_atm
sol.set_equivalence_ratio(equiv_ratio, f'{fuel}:1', 'O2:0.21, N2:0.79')
rho_c = sol.density
yf_c = sol.Y[0]

sol.equilibrate('HP')
temp_eq = sol.T
print(f'Equilibrium temperature = {temp_eq:.3f}')

temp_c = 300
sol.TP = temp_c, ct.one_atm
sol.set_equivalence_ratio(equiv_ratio, f'{fuel}:1', 'O2:0.21, N2:0.79')

ct_flame = ct.FreeFlame(sol, width=8e-2)
ct_flame.set_refine_criteria(ratio=3, slope=0.06, curve=0.12)

ct_flame.transport_model = 'mixture-averaged'
ct_flame.solve(loglevel=0, auto=True);

ct_flame_speed = ct_flame.velocity[0]

print(f"mixture-averaged flame speed = {ct_flame.velocity[0]:7f} m/s")

Equilibrium temperature = 2385.441
mixture-averaged flame speed = 2.325731 m/s


In [4]:
def load_snapshot(case_dir, snapshot_number):

    if snapshot_number:
        add_zeros = 9 - len(str(snapshot_number))
        snap_str = add_zeros*'0' + str(snapshot_number)
    else:
        snap_str = 9*'0'

    db = h5py.File(f'../output/{case_dir}/flame_{snap_str}.h5')
    t, step = db['time/time_and_step'][:]
    x = db['domain/mesh/x'][:]

    density = np.sum(
        db['cons_vars/densities'][:, :],
        axis=0
    )

    u = db['prim_vars/velocity'][:]
    temp = db['prim_vars/temperature'][:]
    mass_frac = db['prim_vars/mass_fractions'][:, :]
    return t, step, x, u, density, temp, mass_frac

In [7]:
er_str = f'{equiv_ratio:.1f}'.replace('.', 'd')
case_dir = f'flame_{fuel_name[fuel]}_er_{er_str}'

if fuel == 'C2H4':
    i_start = 1001
    i_end = 40301
elif fuel == 'H2':
    i_start = 1000
    i_end = 24000 if equiv_ratio == 1.2 else 28000

In [ ]:
fresh_tol = 5
fresh_margin = 20

ts = [] 
xfront = [] 
u_fresh = []
snapshots = []

for n in range(i_start, i_end, 100):
    # Load simulation data
    t, step, x, u, rho, temp, mass_frac = load_snapshot(case_dir, n)

    # Track progress variable front
    c = (temp - temp_c) / (temp_eq - temp_c)
    j = np.argmax(c < 0.5)
    x_f = x[j-1] + (0.5 - c[j-1])*(x[j]-x[j-1])/(c[j]-c[j-1])
    j_margin = min(fresh_margin, len(x)-j-2)
    mask = (np.arange(len(x)) >= j + j_margin) & (temp <= temp_c + fresh_tol)

    # Store results
    ts.append(t)
    xfront.append(x_f)
    u_fresh.append(np.mean(u[mask]))
    snapshots.append(n)

ts = np.asarray(ts)
xfront = np.asarray(xfront)
u_fresh = np.asarray(u_fresh)

# Filter for smoothness
from scipy.signal import savgol_filter
xfs = savgol_filter(xfront, window_length=11, polyorder=2, mode='interp')
vf = np.gradient(xfs, ts)
sl_inst = vf - u_fresh
sl_mean = sl_inst[-100:].mean()
sl_std = sl_inst[-100:].std()

error = np.abs(ct_flame_speed - sl_mean) / ct_flame_speed

print(f"pyro:: flame speed = {sl_mean:.3f} ± {sl_std:.3f} m/s")
print(f'cantera:: flame speed {ct_flame_speed}')
print(f"error: {100*error:.3f}")

# Visualize Flame

In [ ]:
n = 20000
t_1, step_1, x, u_1, density_1, temp_1, mass_frac_1 = load_snapshot(case_dir, n if fuel == 'H2' else n + 1)
omega_1 = pyro_gas.molecular_weights[0] * pyro_gas.get_net_production_rates(
    density_1, temp_1, np.where(mass_frac_1 > 0, mass_frac_1, 0)
)[0]

n = 22000
t_2, step_2, _, u_2, density_2, temp_2, mass_frac_2 = load_snapshot(case_dir, n if fuel == 'H2' else n + 1)
omega_2 = pyro_gas.molecular_weights[0] * pyro_gas.get_net_production_rates(
    density_2, temp_2, np.where(mass_frac_2 > 0, mass_frac_2, 0)
)[0]

n = 23000
t_3, step_3, _, u_3, density_3, temp_3, mass_frac_3 = load_snapshot(case_dir, n if fuel == 'H2' else n + 1)
omega_3 = pyro_gas.molecular_weights[0] * pyro_gas.get_net_production_rates(
    density_3, temp_3, np.where(mass_frac_3 > 0, mass_frac_3, 0)
)[0]

n = 24000
t_4, step_4, _, u_4, density_4, temp_4, mass_frac_4 = load_snapshot(case_dir, n if fuel == 'H2' else n + 1)
omega_4 = pyro_gas.molecular_weights[0] * pyro_gas.get_net_production_rates(
    density_4, temp_4, np.where(mass_frac_4 > 0, mass_frac_4, 0)
)[0]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3))
ax[0].plot(x, temp_1, color='orange')
ax[0].plot(x, temp_2, color='red')
ax[0].plot(x, temp_3, color='crimson')
ax[0].plot(x, temp_4, color='brown')
ax[0].set_xlim(0.03, 0.06)

ax[1].plot(x, mass_frac_1[1], color='orange')
ax[1].plot(x, mass_frac_2[1], color='red')
ax[1].plot(x, mass_frac_3[1], color='crimson')
ax[1].plot(x, mass_frac_4[1], color='brown')
ax[1].set_xlim(0.03, 0.06)

ax[2].plot(x, u_1, color='orange')
ax[2].plot(x, u_2, color='red')
ax[2].plot(x, u_3, color='crimson')
ax[2].plot(x, u_4, color='brown')
ax[2].set_xlim(0.03, 0.06)

plt.tight_layout()
plt.show()